In [1]:
import os
import geopandas as gpd
import pandas as pd
import numpy as np

In [2]:
### Célula para conectar com Google Drive
from google.colab import drive
drive.mount('/content/drive')

if not os.getcwd().endswith('Censo IBGE 2022/Compatibilização'):
    os.chdir('/content/drive/Shareddrives/SIG LabCidade/projetos/Censo IBGE 2022/Compatibilização')

Mounted at /content/drive


In [3]:
nome_compat = 'PIC001022-SSSQQQ-MSP'
nome_A = 'PIC001022'
nome_B = 'SSSQQQ'

## Importar dados de compatibilidade

In [4]:
# Importar perímetros compatíveis
gdf_perim = gpd.read_file(f'{nome_compat}/perimetros_compativeis.gpkg')[['CD_PICSQ','geometry']]
gdf_perim = gdf_perim.to_crs('EPSG:31983')

# Importar matrizes de compatibilidade
matriz_A = pd.read_csv(f'{nome_compat}/matriz_compat_{nome_A}.csv',
                       sep='\t',
                       dtype={'CD_PERIMETRO':str, 'CD_PICSQ':str})
matriz_B = pd.read_csv(f'{nome_compat}/matriz_compat_{nome_B}.csv',
                       sep='\t',
                       dtype={'SSSQQQ':str, 'CD_PICSQ':str})

In [5]:
# Importar dados de PC já agregados
gdf_data = gpd.read_file('2000-2010-2022/perimetros_compativeis_agg.gpkg')

## Reagregação

In [6]:
gdf_data.columns

Index(['CD_PERIMETRO', 'dom_col_2022', 'dom_par_2022', 'dom_pi_2000',
       'dom_pi_2010', 'dom_po_2000', 'dom_po_2010', 'dom_po_2022',
       'dom_po_imput_2022', 'dom_pp_2000', 'dom_pp_2010', 'dom_tot_2000',
       'dom_tot_2010', 'dom_tot_2022', 'med_pop_dom_po_2022',
       'pct_dom_po_imput_2022', 'pop_dom_po_2000', 'pop_dom_po_2022',
       'pop_dom_pp_2000', 'pop_dom_pp_2010', 'pop_tot_2000', 'pop_tot_2010',
       'pop_tot_2022', 'dens_pop_tot_ha_2000', 'dens_dom_tot_ha_2000',
       'dens_dom_po_ha_2000', 'med_pop_dom_pp_2000', 'dens_pop_tot_ha_2010',
       'dens_dom_tot_ha_2010', 'dens_dom_po_ha_2010', 'med_pop_dom_pp_2010',
       'dens_pop_tot_ha_2022', 'dens_dom_tot_ha_2022', 'dens_dom_po_ha_2022',
       'geometry'],
      dtype='object')

In [7]:
# Reagregação
cols = ['dom_col_2022', 'dom_par_2022', 'dom_pi_2000',
       'dom_pi_2010', 'dom_po_2000', 'dom_po_2010', 'dom_po_2022',
       'dom_po_imput_2022', 'dom_pp_2000', 'dom_pp_2010', 'dom_tot_2000',
       'dom_tot_2010', 'dom_tot_2022', 'med_pop_dom_po_2022', 'pop_dom_po_2000', 'pop_dom_po_2022',
       'pop_dom_pp_2000', 'pop_dom_pp_2010', 'pop_tot_2000', 'pop_tot_2010',
       'pop_tot_2022']
agg = gdf_data.merge(matriz_A, on='CD_PERIMETRO', how='right')
agg = agg.pivot_table(index='CD_PICSQ', values=cols, aggfunc='sum').reset_index()

In [8]:
gdf_perim_agg = gdf_perim.merge(agg, on='CD_PICSQ', how='left')

# Densidade populacional
for year in ['2000', '2010', '2022']:
    gdf_perim_agg[f'dens_pop_tot_ha_{year}'] = gdf_perim_agg[f'pop_tot_{year}']/(gdf_perim_agg['geometry'].area/10000)

    # Densidade domiciliar (total)
    gdf_perim_agg[f'dens_dom_tot_ha_{year}'] = gdf_perim_agg[f'dom_tot_{year}']/(gdf_perim_agg['geometry'].area/10000)

    # Densidade domiciliar (ocupados)
    gdf_perim_agg[f'dens_dom_po_ha_{year}'] = gdf_perim_agg[f'dom_po_{year}']/(gdf_perim_agg['geometry'].area/10000)

    # Média de moradores por domicílio
    if year != '2022':
        gdf_perim_agg[f'med_pop_dom_pp_{year}'] = gdf_perim_agg[f'pop_dom_pp_{year}']/gdf_perim_agg[f'dom_pp_{year}']
    else:
        gdf_perim_agg[f'med_pop_dom_po_{year}'] = gdf_perim_agg[f'pop_dom_po_{year}']/gdf_perim_agg[f'dom_po_{year}']

## Exportação

In [9]:
agg.to_csv(f'{nome_compat}/agg_perimetros_{nome_A}-{nome_B}.csv', sep='\t', index=False, decimal=',')

In [ ]:
gdf_perim_agg.to_file(f'{nome_compat}/perimetros_compativeis_agg.gpkg',
                      layer=f'{nome_A}-{nome_B}',
                      driver='GPKG')